# Deep Learning from Audio Signals

### Prepare Training Data: MFCCs from Genre music files and store in JSON file

Rajesh Siraskar | 03-May-2021

In [1]:
import json
import os
import math
import librosa

In [2]:
DATA_PATH = "audio_files/genres_REDUCED/"
JSON_FILE = "audio_files/mfcc_json.json"
SAMPLE_RATE = 22050
TRACK_DURATION = 30 # Seconds - this is known for the music files

SAMPLES_PER_TRACK = SAMPLE_RATE * TRACK_DURATION

In [3]:
def save_mfcc(data_path, json_file, num_mfcc=13, n_fft=2048, hop_length=512, num_segments=10):
    """Extract MFC coefficients from music data-set and save them into a *SINGLE* json file along with labels (genre) 

        :param dataset_path (str): Path to dataset
        :param json_path (str): Path to json file used to save MFCCs
        :param num_mfcc (int): Number of coefficients to extract
        :param n_fft (int): Interval we consider to apply FFT. Measured in # of samples
        :param hop_length (int): Sliding window for FFT. Measured in # of samples
        :param: num_segments (int): Number of segments we want to divide sample tracks into
        :return:
        """

    # JSON dictionary to store mapping, labels, and MFCCs
    data = {
        "mapping": [],  # Semantic label i.e. "classical", "jazz", "blues" etc.
        "labels": [],   # Numeric labels i.e. 0, 1, 2 ... 
        "mfcc": []      # MFCC - these are floats. 13 by default for each segment
    }

    # Note: num_mfcc_vectors_per_segment increases if we reduce the num_segments
    samples_per_segment = int(SAMPLES_PER_TRACK/num_segments)
    num_mfcc_vectors_per_segment = math.ceil(samples_per_segment/hop_length)
    print("\n - SAMPLES_PER_TRACK: {}, num_mfcc_vectors_per_segment: {}".format(SAMPLES_PER_TRACK, num_mfcc_vectors_per_segment))

    # Loop through all genre sub-folder
    #  Use os.walk that is like a crawler on nested sub-folders 
    #  TRICK: 'enumerate' on os.walk allows converting the "GENRE sub-folder" into a NUMBER
    #         and use that as the NUMERIC label!
    for n_label, (dirpath, dirnames, filenames) in enumerate(os.walk(data_path)):

        # Ensure we're processing a genre sub-folder level and not the highest level folder
        if dirpath is not data_path:

            # Save genre label (i.e., sub-folder name) in the mapping
            # Get the semantic genre name from the full file path name's last component
            semantic_label = dirpath.split("/")[-1]
            data["mapping"].append(semantic_label)
            print("\n * Processing genre: \'{}\'".format(semantic_label))

            # Process all audio files in genre sub-dir
            for fname in filenames:
                # Load audio file
                file_path = os.path.join(dirpath, fname)
                signal, sample_rate = librosa.load(file_path, sr=SAMPLE_RATE)

                # Process all segments of audio file
                for n_segment in range(num_segments):

                    # Calculate start and finish sample for current segment
                    start = samples_per_segment*n_segment
                    finish = start+samples_per_segment

                    # Extract mfcc
                    mfcc = librosa.feature.mfcc(signal[start:finish], sample_rate, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
                    
                    # Transpose
                    mfcc = mfcc.T

                    # Store only mfcc feature with expected number of vectors
                    if len(mfcc) == num_mfcc_vectors_per_segment:
                        # Note: mfcc is a numpy array and needs to be converted to list 
                        #  before storing in JSON
                        data["mfcc"].append(mfcc.tolist())
                        data["labels"].append(n_label-1)
                        wav_file = file_path.split("/")[-1] 
                        print("   --- {}, segment:{}".format(wav_file, n_segment+1))

    print("\n\n - MFCC's extracted. Write data to JSON")                        
    # Save MFCCs to json file
    with open(json_file, "w") as fp:
        json.dump(data, fp, indent=4)
        
    print(" - JSON file ready")

In [4]:
save_mfcc(DATA_PATH, JSON_FILE, num_segments=10)


 - SAMPLES_PER_TRACK: 661500, num_mfcc_vectors_per_segment: 130

 * Processing genre: 'blues'
   --- blues\blues.00000.wav, segment:1
   --- blues\blues.00000.wav, segment:2
   --- blues\blues.00000.wav, segment:3
   --- blues\blues.00000.wav, segment:4
   --- blues\blues.00000.wav, segment:5
   --- blues\blues.00000.wav, segment:6
   --- blues\blues.00000.wav, segment:7
   --- blues\blues.00000.wav, segment:8
   --- blues\blues.00000.wav, segment:9
   --- blues\blues.00000.wav, segment:10
   --- blues\blues.00001.wav, segment:1
   --- blues\blues.00001.wav, segment:2
   --- blues\blues.00001.wav, segment:3
   --- blues\blues.00001.wav, segment:4
   --- blues\blues.00001.wav, segment:5
   --- blues\blues.00001.wav, segment:6
   --- blues\blues.00001.wav, segment:7
   --- blues\blues.00001.wav, segment:8
   --- blues\blues.00001.wav, segment:9
   --- blues\blues.00001.wav, segment:10

 * Processing genre: 'classical'
   --- classical\classical.00000.wav, segment:1
   --- classical\clas

Understand Saved Data
====================

- Elements saved
- Dimensions of data
- Later in training this load_data function is used and reshape data for training


**3-D array dimensions dim.**: 
- Given: hop_length=512; n_classes = 4 genres; training data is (only) 2 wav.files per genre 
- dimension 1: num_segments x n_classes x .wav-files-per-class 
- dimension 2: num_mfcc_vectors_per_segment = math.ceil(samples_per_segment/hop_length)
- dimension 3: num_mfcc

```
Example: 

- hop_length=512; n_classes = 4 genres; .wav files per genre = 2
- num_segments = 10; num_mfcc=13; SAMPLE_RATE=22050; TRACK_DURATION=30 s
- SAMPLES_PER_TRACK = SAMPLE_RATE * TRACK_DURATION = 22050 x 30
- samples_per_segment = int(SAMPLES_PER_TRACK/num_segments) = int(22050 x 30/10) = 66150.0

1. dim-1: 10 x 4 x 2 = 80 
2. dim-2: math.ceil(samples_per_segment/hop_length) = 66150/512 = 129.12 -> ceil -> 130
3. dim-3: 13

Therefore X.data.shape = (80, 130, 13) and y.data.shape: (80,)
```

**Input layer**: 
```
    # input = 2D: MFCC for each segment, and each mfcc is a vector 
    #   over an interval = hop-length
    #   first-dim: interval = inputs.shape[1]
    #   second-dim: mfcc = inputs.shape[2]
    #   NOTE: inputs.shape[0] is the segment number and we are not passing that
    keras.layers.Flatten(input_shape=(X.shape[1], X.shape[2])),
```

In [5]:
import json
import numpy as np

def load_data (json_file):
    with open(json_file, "r") as fp:
        data = json.load(fp)
        
        # Note: mfcc was converted from a numpy array to list before storing in JSON
        #  Need convert back to numpy array
        X = np.array(data["mfcc"])
        y = np.array(data["labels"])
        
    return X, y

In [6]:
### 1. Load data
X, y = load_data (json_file = JSON_FILE)

In [7]:
print("num_mfcc=13; num_segments=10; num_mfcc_vectors_per_segment: 130\n")

print("X.data.shape: ", X.data.shape)
print("y.data.shape: ", y.data.shape)

num_mfcc=13; num_segments=10; num_mfcc_vectors_per_segment: 130

X.data.shape:  (80, 130, 13)
y.data.shape:  (80,)


In [39]:
# print 1st element of the 3 D array
X[0][0][0]

-212.68553161621094

In [50]:
X, y = load_data (json_file = JSON_FILE)

print("num_mfcc=8; num_segments=4; num_mfcc_vectors_per_segment: 323\n")

print("X.data.shape: ", X.data.shape)
print("y.data.shape: ", y.data.shape)

num_mfcc=8; num_segments=4; num_mfcc_vectors_per_segment: 323

X.data.shape:  (32, 323, 8)
y.data.shape:  (32,)


In [53]:
X, y = load_data (json_file = JSON_FILE)

print("num_mfcc=10; num_segments=6; num_mfcc_vectors_per_segment: 216\n")

print("X.data.shape: ", X.data.shape)
print("y.data.shape: ", y.data.shape)

num_mfcc=10; num_segments=6; num_mfcc_vectors_per_segment: 216

X.data.shape:  (48, 216, 10)
y.data.shape:  (48,)


In [60]:
samples_per_segment = int(SAMPLES_PER_TRACK/10)
num_mfcc_vectors_per_segment = math.ceil(samples_per_segment/512)
samples_per_segment, num_mfcc_vectors_per_segment

(66150, 130)

In [77]:
X.shape[1], X.shape[2]

(130, 13)